In [1]:
# Loading libraries:

from datetime import datetime
import numpy as np
from numpy import random
import pandas as pd
import glob
import os

In [2]:
# Compiling data from sub-datasets:

file_path = r'/Users/aloysiusteng/Documents (local drive)/CS610/Project/data/resale-flat-prices'
file_list = os.listdir(file_path)
csv_files = list(filter(lambda f: f.endswith('.csv'), file_list))
csv_files

['resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv',
 'resale-flat-prices-based-on-approval-date-1990-1999.csv',
 'resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv',
 'resale-flat-prices-based-on-approval-date-2000-feb-2012.csv',
 'resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv']

In [3]:
#concatinate sales data
df_resale = pd.concat([pd.read_csv(file_path + '/' + f) for f in csv_files], ignore_index=True)

In [5]:
# Get resale data and process time column

df_resale['date'] = pd.to_datetime(df_resale['month'], format = '%Y-%m')
df_resale['remaining_lease'] = 99 - (df_resale['date'].dt.year - df_resale['lease_commence_date'])
df_resale["storey"] = (df_resale["storey_range"].map(lambda x: int(str(x)[6:]))+df_resale["storey_range"].map(lambda x: int(str(x)[:2])))/2
df_resale['storey']=df_resale['storey'].astype('int')
df_resale['month_count'] = (df_resale['date'].dt.year - 1999)*12 + (df_resale['date'].dt.month - 10) #starting from Oct 1999
df_resale = df_resale[df_resale['month_count'] >= 0]


df_resale.tail(1)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,date,storey,month_count
894020,2016-12,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,10 TO 12,164.0,Multi Generation,1987,735000.0,70,2016-12-01,11,206


In [7]:
# load coordinate data
hdb_geo = pd.read_csv('data/geo-hdbs.csv')
phpcs_geo = pd.read_csv('data/geo-phpcs.csv')
schools_geo = pd.read_csv('data/geo-schools.csv')
supermarkets_geo = pd.read_csv('data/geo-supermarkets.csv')
hawkercentres_geo = pd.read_csv('data/geo-hawkercentres.csv')
mrtlrt_geo = pd.read_csv('data/geo-mrtlrtstations.csv')
mrtlrt_geo['OPENING_DATE'] = pd.to_datetime(mrtlrt_geo['OPENING_DATE'], format="%d/%m/%Y")

city_lat = 1.2841256
city_lon = 103.8514617

In [8]:
# join blk and street name
df_resale['ADDRESS'] = df_resale['block'] + ' ' + df_resale['street_name']
df_resale.tail(2)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease,date,storey,month_count,ADDRESS
894019,2016-12,YISHUN,EXECUTIVE,325,YISHUN CTRL,01 TO 03,146.0,Maisonette,1988,575000.0,71,2016-12-01,2,206,325 YISHUN CTRL
894020,2016-12,YISHUN,MULTI-GENERATION,666,YISHUN AVE 4,10 TO 12,164.0,Multi Generation,1987,735000.0,70,2016-12-01,11,206,666 YISHUN AVE 4


In [9]:
#create haversine function
import math

def haversine(lat1, lon1, lat2, lon2):
    
    # Coordinates in decimal degrees (e.g. 2.89078, 12.79797)

    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi_1) * math.cos(phi_2) * math.sin(delta_lambda / 2.0) ** 2
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    meters = R * c  # output distance in meters
    km = meters / 1000.0  # output distance in kilometers

    return round(km, 3)

In [10]:
# get closes amenities
from tqdm import tqdm

dist_phpcs = []
dist_schools = []
dist_supermarkets = []
dist_hawkercentres = []
dist_city = []

for _, i in tqdm(hdb_geo.iterrows()):
    phpcs=[]
    schools =[]
    supermarkets = []
    hawkercentres = []
    for _, j in phpcs_geo.iterrows():
        phpcs.append(haversine(i["LATITUDE"], i["LONGITUDE"], j["LATITUDE"], j["LONGITUDE"]))
    dist_phpcs.append(min(phpcs))
    for _, j in schools_geo.iterrows():
        schools.append(haversine(i["LATITUDE"], i["LONGITUDE"], j["LATITUDE"], j["LONGITUDE"]))
    dist_schools.append(min(schools))
    for _, j in supermarkets_geo.iterrows():
        supermarkets.append(haversine(i["LATITUDE"], i["LONGITUDE"], j["LATITUDE"], j["LONGITUDE"]))
    dist_supermarkets.append(min(supermarkets))
    for _, j in hawkercentres_geo.iterrows():
        hawkercentres.append(haversine(i["LATITUDE"], i["LONGITUDE"], j["LATITUDE"], j["LONGITUDE"]))
    dist_hawkercentres.append(min(hawkercentres))
    dist_city.append(haversine(i["LATITUDE"], i["LONGITUDE"], city_lat, city_lon))

9714it [14:37, 11.07it/s]


In [11]:
# add distance to closes amenities
hdb_geo["dist_phpcs"] = dist_phpcs
hdb_geo["dist_schools"] = dist_schools
hdb_geo["dist_supermarkets"] = dist_supermarkets
hdb_geo["dist_hawkercentres"] = dist_hawkercentres
hdb_geo["dist_city"] = dist_city

In [24]:
# merge with geo data
all_data = df_resale.merge(
    hdb_geo[["ADDRESS","LATITUDE", "LONGITUDE", "dist_phpcs","dist_schools", "dist_supermarkets", "dist_hawkercentres", "dist_city"]],
    how="left",
    on="ADDRESS"
    )

In [17]:
# get distance to closes train station

dist_mrtlrt = []

for _, i in tqdm(all_data.iterrows()):
    dist=[]
    for _, j in mrtlrt_geo[mrtlrt_geo["OPENING_DATE"]<=i["date"]].iterrows():
        dist.append(haversine(i["LATITUDE"], i["LONGITUDE"], j["LATITUDE"], j["LONGITUDE"]))
    dist_mrtlrt.append(min(dist))

dist_mrtlrt

616778it [2:01:54, 84.32it/s] 


[1.227,
 0.389,
 0.695,
 1.233,
 1.019,
 0.738,
 1.351,
 0.759,
 1.249,
 0.528,
 0.849,
 1.075,
 0.925,
 1.042,
 0.896,
 0.685,
 1.081,
 0.892,
 0.984,
 1.408,
 0.701,
 0.619,
 1.292,
 0.946,
 1.019,
 0.954,
 0.722,
 0.944,
 0.568,
 0.607,
 0.607,
 0.592,
 0.465,
 0.946,
 0.542,
 0.681,
 0.746,
 0.696,
 0.987,
 1.146,
 0.719,
 0.678,
 0.958,
 0.681,
 1.204,
 0.746,
 0.321,
 0.552,
 0.632,
 0.722,
 1.204,
 1.075,
 0.836,
 0.457,
 0.676,
 0.414,
 0.899,
 0.925,
 0.465,
 0.729,
 0.32,
 1.408,
 1.02,
 0.5,
 0.461,
 1.094,
 0.461,
 0.371,
 0.497,
 1.293,
 0.951,
 1.33,
 0.585,
 0.887,
 0.86,
 0.722,
 0.776,
 0.589,
 0.883,
 0.889,
 0.758,
 1.119,
 0.714,
 0.371,
 0.91,
 1.346,
 0.574,
 0.676,
 0.951,
 0.187,
 0.548,
 0.827,
 0.489,
 0.316,
 0.209,
 1.122,
 0.727,
 0.461,
 0.414,
 0.629,
 0.414,
 0.445,
 0.387,
 0.271,
 0.414,
 1.136,
 0.393,
 0.51,
 0.445,
 1.657,
 0.232,
 0.646,
 0.596,
 0.694,
 0.694,
 0.691,
 0.387,
 0.346,
 0.271,
 1.57,
 1.408,
 1.57,
 0.918,
 0.879,
 0.918,
 1.197,
 1

In [25]:
# merge with geo data
all_data['dist_mrtlrt'] = dist_mrtlrt

In [26]:
# Load interest rate:
int_rate = pd.read_excel('data/sg int.xlsx')
int_rate['Date'] = pd.to_datetime(int_rate['Date'])

int_rate = int_rate.sort_values(by='Date', ascending=True, ignore_index=True)
int_rate.head(2)

,Date,Ask Price
0,1999-09-30,2.47917
1,1999-12-31,2.83333


In [27]:
# merge with interest rate data
all_data = all_data.sort_values(by='date', ascending=True, ignore_index=True)

all_data = pd.merge_asof(all_data, 
              int_rate[["Date", "Ask Price"]],
              left_on='date',
              right_on='Date',
              direction='backward')

all_data = all_data.rename(columns={"Ask Price": "int_rate"})
all_data = all_data.drop('Date', axis=1)

In [28]:
# remove columns

all_data = all_data.drop(['date','month', 'block', 'storey_range', 'street_name', 'lease_commence_date', 'ADDRESS', 'LATITUDE', 'LONGITUDE'], axis=1)

In [29]:
# clean data

all_data['flat_type'] = all_data['flat_type'].replace('MULTI-GENERATION', 'MULTI GENERATION', regex = True)

for i in ['town', 'flat_type', 'flat_model']:
    all_data[i] = all_data[i].map(lambda x: x.replace(' ', '_'))
    all_data[i] = all_data[i].map(lambda x: x.lower())

In [30]:
# reorder columns
order = ['town',
 'flat_type',
 'flat_model',
 'month_count',
 'floor_area_sqm',
 'storey',
 'remaining_lease',
 'dist_phpcs',
 'dist_schools',
 'dist_supermarkets',
 'dist_hawkercentres',
 'dist_mrtlrt',
 'dist_city',
 'int_rate',
 'resale_price']

all_data = all_data.reindex(columns=order)

In [34]:
# save non-normalised data frame to CSV

all_data.to_csv('data/complete_data_unscaled.csv', index=False)

In [35]:
all_data.head(10)

,town,flat_type,flat_model,month_count,floor_area_sqm,storey,remaining_lease,dist_phpcs,dist_schools,dist_supermarkets,dist_hawkercentres,dist_mrtlrt,dist_city,int_rate,resale_price
0,hougang,4_room,new_generation,0,93.0,8,85,0.336,0.506,0.582,0.847,4.489,10.024,2.47917,227000.0
1,queenstown,3_room,standard,0,66.0,5,71,0.297,0.340,0.528,0.272,0.579,6.096,2.47917,131000.0
2,queenstown,3_room,standard,0,52.0,5,73,0.154,0.074,0.338,0.182,0.376,6.023,2.47917,122000.0
3,queenstown,3_room,standard,0,56.0,2,73,0.170,0.214,0.402,0.159,0.450,6.091,2.47917,128000.0
4,queenstown,4_room,improved,0,82.0,14,77,0.108,0.759,0.105,0.207,0.295,7.540,2.47917,290000.0
5,queenstown,4_room,improved,0,89.0,2,73,0.343,0.638,0.491,0.333,0.675,5.824,2.47917,233000.0
6,queenstown,4_room,improved,0,86.0,11,74,0.748,0.504,0.353,0.908,1.305,8.545,2.47917,275000.0
7,queenstown,4_room,improved,0,88.0,14,74,0.170,1.212,0.282,0.242,0.792,7.113,2.47917,296000.0
8,queenstown,4_room,improved,0,88.0,8,75,0.173,1.176,0.162,0.177,0.470,6.957,2.47917,318000.0
9,queenstown,4_room,improved,0,88.0,5,75,0.181,1.113,0.068,0.134,0.399,7.050,2.47917,305000.0


In [36]:
# scale data

scale = {}
for i in range(3,len(all_data.columns)-1):
    lowest = min(all_data[all_data.columns[i]])
    highest = max(all_data[all_data.columns[i]])
    all_data[all_data.columns[i]] = all_data[all_data.columns[i]].map(lambda x: (x-lowest)/(highest-lowest))
    scale[all_data.columns[i]] = [lowest, highest]

In [37]:
# save non-normalised data frame to CSV

all_data.to_csv('data/complete_data.csv', index=False)

In [38]:
# convert 'scale' to dataframe and save as CSV

scale = pd.DataFrame.from_dict(scale)
scale = scale.rename(index={0: 'lowest', 1: 'highest'})
scale['remaining_lease'] = scale['remaining_lease'].astype('int')
scale['month_count'] = scale['month_count'].astype('int')

scale.to_csv('data/scale.csv', index=False)

In [39]:
hdb_geo.to_csv('data/combined_geo', index=False)

In [2]:
all_data['flat_model'].unique()

NameError: name 'all_data' is not defined